# Solving Burgers with Transfer Learning

## Import Relevant Packages

In [1]:
import numpy as np
import sciann as sn
import tensorflow as tf

---------------------- SCIANN 0.6.1.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 


## Get Inputs and Neural Network Approximations

In [2]:
# Define the necessary inputs using Variable 
x = sn.Variable('x') # space
t = sn.Variable('t') # time
d = sn.Variable('d') # boundary perturbation

v_low = 0.05 # viscosity for low fidelity model
v_high = .001 # viscosity for high fidelity model

# Get neural network predictions for u for the low fidelity model
u = sn.Functional('u', [d,t,x], 8*[20], 'tanh') 

## Set up Optimization Problem with Boundary Conditions

In [3]:
from sciann.utils.math import diff, sign, sin

TOL = 0.001
# Low fidelity partial differential equation
L1 = diff(u, t) - v_low * diff(u, x, order=2)

# Low fidelity boundary conditions
L2 = (1 + sign(0-t)) * (u - 1 + (1+x)*(1 + d/2))
L3 = (1 + sign(-1-x)) * u * (1 + d)
L4 = (1 + sign(x-1)) * u

## Build PINN

In [4]:
m = sn.SciModel([d,t,x], [L1, L2, L3, L4])

## Get Training Data

In [5]:
x_data, t_data, d_data = np.meshgrid(
    np.linspace(-1, 1, 10),
    np.linspace(0, 12, 10),
    np.linspace(0, 0.1, 10))

## Train PINN

In [6]:
h = m.train(
    [d_data, t_data, x_data], 
    4*['zero'], 
    learning_rate=0.001, 
    epochs=10000,
    verbose = 0)


Total samples: 1000 
Batch size: 64 
Total batches: 16 


Epoch 02403: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 04554: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 05554: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 06554: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 07554: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 08554: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 09554: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.


In [7]:
low_fidelity_layers = u.layers

# Transfer Learning

## Get 'u' and Information Learned from the Low Fidelity Model to use in the new PINN

## Set up Optimization Problem and Boundary Conditions for High Fidelity

In [8]:
# High fidelity partial differential equation
L5 = diff(u, t) + u * diff(u, x) - v_high * diff(u, x, order=2)

# High fidelity boundary conditions (same as low fidelity model)
x_min = -1
x_max = 1
L6 = (1 + sign(0-t)) * (u - 1 + (1+x)*(1 + d/2))
L7 = (1 + sign(-1-x)) * u * (1 + d)
L8 = (1 + sign(x-1)) * u

## Freeze Layers of the Low Fidelity Model

In [9]:
# Prevent all they layers except the last one from being retrained so the low fidelity model information is not
# lost as well as to prevent overfitting
for layer in u.layers[:-3]:
    if layer.count_params() > 0:
        layer.trainable = False

## Build the New Multi-Fidelity PINN

In [12]:
m2 = sn.SciModel([d,t,x], [L5, L6, L7, L8])

## Train PINN

In [13]:
# Implement an early stopping callback to prevent the model from training when loss increases
h2 = m2.train(
    [d_data, t_data, x_data], 
    4*['zero'], 
    learning_rate=0.001, 
    epochs=1000,
    verbose = 0)


Total samples: 1000 
Batch size: 64 
Total batches: 16 


Epoch 00160: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00309: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 00409: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 00509: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00609: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 00709: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 00809: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00909: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.


In [14]:
multi_fidelity_layers = u.layers